In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import anndata as ad
    
import matplotlib.pyplot as plt

# Load data

## Load abc data

In [5]:
abc_root = Path('//allen/programs/celltypes/workgroups/rnaseqanalysis/mFISH/meghanturner/abc_atlas/abc_atlas_data/')
brain_id = 'C57BL6J-638850'
version='20230830'

In [6]:
# load in the 'obs' cluster_annotations + parcellation_annotation metadata
ccf_md_df = pd.read_csv(abc_root/f'metadata/MERFISH-{brain_id}-CCF/{version}/views/cell_metadata_with_parcellation_annotation.csv', 
                        dtype={'cell_label':str}, 
                        index_col=0)

# flip y coords so the sections don't display upside down
flip_y=True
if flip_y:
    ccf_md_df[['y_section', 'y_reconstructed']] *= -1

In [8]:
ccf_md_df

## Load MK's ccf coordinate csv

In [26]:
mk_root_dir = Path("/allen/programs/celltypes/workgroups/rnaseqanalysis/mFISH/michaelkunst/MERSCOPES/mouse/atlas/mouse_638850")
ccf_coord_file = "alignment/ccf_cells_20230515.csv"
# ccf_coord_path = root_dir / ccf_coord_file

In [3]:
ccf_coord_df = pd.read_csv(mk_root_dir / ccf_coord_file, 
                           dtype={'cell_id':str}, 
                           index_col=0)

In [4]:
ccf_coord_df

In [89]:
bin_coord_df = pd.read_csv(mk_root_dir/'alignment/ccf_cells_20221201.csv',
                           dtype={'_index':str},
                           index_col=0)

In [90]:
bin_coord_df

## Load in other MK's files

### with "ccf3D" coords?

In [27]:
cirro_CCF_ad = ad.read_h5ad(mk_root_dir/'cirro_folder/atlas_brain_638850_CCF.h5ad', backed='r')

In [28]:
cirro_CCF_ad

### with "global3D" & "corrected" coords

In [99]:
cirro_anno_ad = ad.read_h5ad(mk_root_dir/'cirro_folder/mouse_633850_new_region_anno.h5ad', backed='r')

In [100]:
cirro_anno_ad

### metadata csv with maybe some coords?

In [77]:
md_csv = pd.read_csv(mk_root_dir/'analysis/data/metadata_638850.csv',
                     dtype={'cell_id':str}, 
                     index_col=1)

In [78]:
md_csv

# Plot coordinates

## section coords

In [29]:
# sorted(ccf_md_df['z_section'].unique())
z_pos = 6.4
curr_sec = ccf_md_df[ccf_md_df['z_section']==z_pos]
sec_cell_labels = curr_sec.index

In [93]:
ccf_coord_df.index

In [92]:
sec_ccf_coord_df = ccf_coord_df[ccf_coord_df.index.intersection(sec_cell_labels)]
sec_ccf_coord_df

In [30]:
# ref_labels = sec_cell_labels
sec_adata = cirro_CCF_ad[cirro_CCF_ad.obs_names.intersection(sec_cell_labels)].to_memory()

In [101]:
sec_cirro_anno_ad = cirro_anno_ad[cirro_anno_ad.obs_names.intersection(sec_cell_labels)].to_memory()

In [102]:
sec_cirro_anno_ad

In [112]:
fig, axes = plt.subplots(1,3,figsize=(16,4))
axes = axes.ravel()


i=0
axes[i].scatter(curr_sec['x_section'], 
                curr_sec['y_section'],
                s=0.01, c ='green')
axes[i].axis('equal')
axes[i].set_title('x/y_section')


i=1
axes[i].scatter(sec_adata.obsm['spatial_rotated'][:,0],
                sec_adata.obsm['spatial_rotated'][:,1],
                s=0.01, c='grey')
axes[i].axis('equal')
axes[i].set_title('spatial_rotated')

# i=2
# axes[i].scatter(curr_sec['x_section']*1000, 
#                 (curr_sec['y_section']+5)*1000,
#                 s=0.01, c ='red')
# axes[i].axis('equal')


i=2
axes[i].scatter(sec_cirro_anno_ad.obs['corrected_x'],
                -sec_cirro_anno_ad.obs['corrected_y'],
                s=0.01, c='blue')
axes[i].axis('equal')
axes[i].set_title('corrected_x/y')

In [113]:
fig = plt.figure()
ax = plt.gca()

ax.scatter(curr_sec['x_section'], 
                curr_sec['y_section'],
                s=0.01, c ='red')

ax.scatter(sec_cirro_anno_ad.obs['corrected_x']/1000,
            -sec_cirro_anno_ad.obs['corrected_y']/1000,
            s=0.01, c='green')
plt.axis('equal')

In [67]:
fig = plt.figure()
ax = plt.gca()

ax.scatter((sec_adata.obsm['spatial_rotated'][:,0]/1001)+0.6,
           (sec_adata.obsm['spatial_rotated'][:,1]/1001)-5.2,
           s=0.01, c='grey')

ax.scatter(curr_sec['x_section'], 
           curr_sec['y_section'],
           s=0.01, c ='blue')

plt.axis('equal')

## ccf vs reconstructed coords

In [94]:
sec_adata.obsm['ccf3D']

In [117]:
fig, axes = plt.subplots(1,3,figsize=(16,6))
axes = axes.ravel()

i=0
axes[i].scatter(sec_adata.obsm['ccf3D'][:,0],
                sec_adata.obsm['ccf3D'][:,1],
                s=0.01, c='grey')
# axes[i].axis('equal')
axes[i].set_title('ccf3D')

i=1
axes[i].scatter(curr_sec['x_ccf'], 
                -curr_sec['y_ccf'],
                s=0.01, c ='r')
# axes[i].axis('equal')
axes[i].set_title('x/y_ccf')


i=2
axes[i].scatter(curr_sec['x_reconstructed'], 
                curr_sec['y_reconstructed'],
                s=0.01, c ='blue')
axes[i].axis('equal')
axes[i].set_title('x/y_reconstructed')



# i=3
# axes[i].scatter(sec_bin_coord_df['bin_x'], 
#                 sec_bin_coord_df['bin_y'],
#                 s=0.01, c ='green')
# axes[i].axis('equal')